# Exercise 01:  Minimal high-level UQ example

This notebook demonstrates high-level UQ with CUQIpy on a 1D Deconvolution test problem.

## Learning objectives of this notebook:
- Get acquainted with the CUQIpy components needed to specify a Bayesian inverse problem.
- Run a high-level UQ analysis of an inverse problem.

## Table of contents: 
* [1. UQ in five lines!](#UQ5)
* [2. Loading a test problem](#TestProblem)
* [3. Specifying and solving a Bayesian inverse problem ](#Bayesian)
* [4. ★ Changing the prior](#ChangingPrior)
* [5. ★ Gibbs sampling for hierarchical problems](#Gibbs)

*The last two sections marked by "★" are optional - try running through sections 1, 2, 3 before attempting the exercises, and before sections 4 and 5.*

First we import any python packages we need, here simply NumPy to handle array computations and matplotlib for plotting:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Then we import the functionality we need from CUQIpy:

In [ ]:
import sys
sys.path.append("../../CUQIpy/")

from cuqi.distribution import JointDistribution, Gaussian, GaussianCov, GaussianPrec, Gamma, Laplace_diff, Cauchy_diff
from cuqi.problem import BayesianProblem
from cuqi.testproblem import Deconvolution1D, Deconvolution2D

## 1. UQ in a few lines of code!   <a class="anchor" id="UQ5"></a> 

In just a few lines of CUQIpy code we can:
- load a forward model and data from a 1D deconvolution test problem,
- specify distributions for prior and data,
- formulate a Bayesian inverse problem with observed data, and
- run a UQ analysis.

The 1D deconvolution test problem is a linear inverse problem with additive noise:

$$ \mathbf{y} = \mathbf{A}\mathbf{x}+\mathbf{e},$$
where 
- $\mathbf{A}\in\mathbb{R}^{n\times n}$, is a blurring operator, 
- $\mathbf{x}\in\mathbb{R}^n$ is an unknown 1D signal of length $n$ to be determined,
- $\mathbf{y}\in\mathbb{R}^n$ is an observed blurred and noisy signal, and
- $\mathbf{e}\in\mathbb{R}^n$ is additive noise.

We load in the forward model `A` and observed data `y_data` from the test problem:

In [ ]:
n = 50
A, y_data, probInfo = Deconvolution1D.get_components(dim=n, phantom="Square")

We assume (as a prior) that the elements of $\mathbf{x}$ are i.i.d. zero-mean Gaussian with variance $\sigma_x^2$ or equivalently *precision* $d_x = 1/\sigma_x^2$, and the additive noise is also i.i.d. Gaussian with zero mean and variance $\sigma_e^2$, i.e., precision $d_e = 1/\sigma_e^2$,
$$
\begin{align}
\mathbf{x}&\sim\mathcal{N}(\mathbf{0},d_x^{-1}\mathbf{I}_n), \\
\mathbf{e}&\sim\mathcal{N}(\mathbf{0},d_e^{-1}\mathbf{I}_n)
\end{align}
$$

From the distribution of $\mathbf{e}$ and the linear model we can write up this with the distribution of $\mathbf{y}$ instead:
$$
\begin{align}
\mathbf{x}&\sim\mathcal{N}(\mathbf{0},d_x^{-1}\mathbf{I}_n), \\
\mathbf{y}&\sim\mathcal{N}(\mathbf{Ax},d_e^{-1}\mathbf{I}_n).
\end{align}$$

which in CUQIpy can be written as follows for known precisions:

In [ ]:
d_x_true = 100
d_e_true = 400
x = GaussianCov(mean=np.zeros(n), cov=1/d_x_true)
y = GaussianCov(mean=A@x, cov=1/d_e_true)

We can carry out a UQ analysis given observed data $\mathbf{y}=\mathbf{y}^\mathrm{data}$ by:

In [ ]:
BP = BayesianProblem(y,x).set_data(y=y_data)
BP.UQ(exact=probInfo.exactSolution)

If the precisions are not known, we can include both as parameters to be estimated in the inverse problem. We extend the Bayesian problem by assuming the precisions follow Gamma distributions in a hierarchical problem:
$$
\begin{align*}
\delta_x   &\sim \mathrm{Gamma}(1, 10^{-2}), \\
\delta_e   &\sim \mathrm{Gamma}(1, 10^{-2}), \\
\mathbf{x} &\sim \mathcal{N}(\mathbf{0},d_x^{-1}\mathbf{I}_n), \\
\mathbf{y} &\sim \mathcal{N}(\mathbf{Ax},d_e^{-1}\mathbf{I}_n).
\end{align*}
$$

In [ ]:
d_x = Gamma(1, 0.01)
d_e = Gamma(1, 0.01)
x = GaussianCov(np.zeros(n), cov=lambda d_x : 1/d_x)
y = GaussianCov(A@x, cov=lambda d_e : 1/d_e)

UQ analysis given observed data $\mathbf{y}=\mathbf{y}^\mathrm{data}$, now on the extended problem using a Gibbs sampler:

In [ ]:
BP = BayesianProblem(y, x, d_x, d_e).set_data(y=y_data)
BP.UQ(exact={"x":probInfo.exactSolution, "d_e":d_e_true})

The plots for the hyperparameters $\delta_x$ and $\delta_e$ are called trace plots. They show the density estimate (left) and chain (right).

In the following sections of this notebook we break it down and take a slightly closer look at each step. The subsequent notebooks provide additional details.

## 2. Loading a test problem  <a class="anchor" id="TestProblem"></a> 

The 1D deconvolution test problem is a linear inverse problem with additive noise:

$$ \mathbf{y} = \mathbf{A}\mathbf{x}+\mathbf{e},$$
where 
- $\mathbf{A}\in\mathbb{R}^{n\times n}$, is a blurring operator, 
- $\mathbf{x}\in\mathbb{R}^n$ is an unknown 1D signal of length $n$ to be determined,
- $\mathbf{y}\in\mathbb{R}^n$ is an observed blurred and noisy signal, and
- $\mathbf{e}\in\mathbb{R}^n$ is additive noise.

We load in the forward model `A` and observed data `y_data` from the test problem:

In [ ]:
A, y_data, probInfo = Deconvolution1D.get_components(dim=n, phantom="Square")

Here we use the variable name `A` to match the mathematics. But `A` is a CUQIpy model, not a matrix.

Note that the test problem can be configured (e.g. other phantoms, noise types and level etc.) by means of the inputs, see `help(Deconvolution1D)` for details.

Several other test problems are available (click links to see online docs):
- [Deconvolution1D](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.testproblem/cuqi.testproblem.Deconvolution1D.html#cuqi.testproblem.Deconvolution1D): 1D periodic deconvolution problem.
- [Deconvolution2D](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.testproblem/cuqi.testproblem.Deconvolution2D.html#cuqi.testproblem.Deconvolution2D): 2D deconvolution (deblurring) problem.
- [Heat_1D](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.testproblem/cuqi.testproblem.Heat_1D.html):  Heat equation PDE problem.
- [Poisson_1D](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.testproblem/cuqi.testproblem.Poisson_1D.html): Poisson equation PDE problem.
- [Abel_1D](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.testproblem/cuqi.testproblem.Abel_1D.html):  Abel equation PDE (1D rotationally symmetric CT) problem. 

These are imported as follows:

In [ ]:
from cuqi.testproblem import Deconvolution2D, Heat_1D, Poisson_1D, Abel_1D

and their calling signature is the same as for the `Deconvolution1D` problem, e.g. 
```
model, data, probInfo = Heat_1D.get_components()
```
Input arguments vary and default values are provided if left empty. Instead of checking the onlinen docs, calling help of each testproblem, e.g., `help(Heat_1D)` will also describe the test problem and the inputs it accepts

For now, proceeding with the `Deconvolution1D` test problem, we take a look at the model and see that it is a CUQIpy LinearModel:

In [ ]:
A

We take a look at the data:

In [ ]:
y_data

The data is a `CUQIarray`, which is a normal NumPy array further equipped with a few utilities, such as Geometry, which allows us to do plotting conveniently:

In [ ]:
y_data.plot()
plt.title("Data");

The last thing returned by the test problem was `probInfo` which contains additional information about the test problem, typically it includes the exact solution (phantom) and the exact data. We take a look at both:

In [ ]:
probInfo

In [ ]:
probInfo.exactSolution.plot()
plt.title("Exact solution");

In [ ]:
probInfo.exactData.plot()
plt.title("Exact data");

Since `CUQIarray` is a NumPy array (technically subclassed from NumPy ndarray), we can do all computations that NumPy admits and still get a `CUQIarray`, for example take the difference between the data and exact data and call the plot method:

In [ ]:
(y_data-probInfo.exactData).plot()
plt.title("Noise");

## 3. Specifying and solving a Bayesian inverse problem  <a class="anchor" id="Bayesian"></a> 

We assume (as a prior) that the elements of $\mathbf{x}$ are i.i.d. zero-mean Gaussian with variance $\sigma_x^2$ or equivalently *precision* $d_x = 1/\sigma_x^2$, and the additive noise is also i.i.d. Gaussian with zero mean and variance $\sigma_e^2$, i.e., precision $d_e = 1/\sigma_e^2$,
$$
\begin{align}
\mathbf{x}&\sim\mathcal{N}(\mathbf{0},d_x^{-1}\mathbf{I}_n), \\
\mathbf{e}&\sim\mathcal{N}(\mathbf{0},d_e^{-1}\mathbf{I}_n)
\end{align}
$$

From the distribution of $\mathbf{e}$ and the linear model we can write up this with the distribution of $\mathbf{y}$ instead:
$$
\begin{align}
\mathbf{x}&\sim\mathcal{N}(\mathbf{0},d_x^{-1}\mathbf{I}_n), \\
\mathbf{y}&\sim\mathcal{N}(\mathbf{Ax},d_e^{-1}\mathbf{I}_n).
\end{align}$$

We can specify these distributions in CUQIpy as follows. For $\mathbf{x}$ with a particular fixed choice of $\mathbf{\sigma_x}$:

In [ ]:
d_x_true = 100
x = GaussianCov(mean=np.zeros(n), cov=1/d_x_true)

We create the Gaussian distribution with the model applied to `x` as mean and a choice of fixed standard deviation:

In [ ]:
d_e_true = 400
y = GaussianCov(A@x, cov=1/d_e_true)

The two distributions are combined in a Bayesian inverse problem:

In [ ]:
BP = BayesianProblem(y, x)
print(BP)

This is essentially a joint distribution over `x` and `y`. The last step is to provide the observed data for `y`:

In [ ]:
BP.set_data(y=y_data)

The "completely non-expert approach" to solving (more detailed approaches described in later notebooks) is to simply run the `UQ` method. 

The `UQ` method looks at the components of the inverse problem, chooses a suitable sampler, samples the posterior and presents results visually:

In [ ]:
BP.UQ()

To compare with the exact solution (if available) one can pass it as an input, and we can also return the posterior samples:

In [ ]:
samples = BP.UQ(exact=probInfo.exactSolution)

Having returned the samples allows us to further investigate the posterior in addition to the prefined analysis done by `.UQ()`, for example we can plot 5 randomly chosen samples:

In [ ]:
samples.plot();

## Try yourself (optional):  
- Play with the parameters of the Gaussian prior to see if a better solution can be obtained.
- Change phantom to another of the options provided by the TestProblem. Hint: use `help(Deconvolution1D)` to see which phantoms are available.
- Try changing to the `Deconvolution2D` test problem and run the same code.

Go to the top of section 2 to modify the code or add the relevant code in the cell below.

## 4. ★ Changing the prior <a class="anchor" id="ChangingPrior"></a> 

It is straightforward to change components of the BayesianProblem. For example if we want to experiment with a different prior we can easily swap it out.

We specify a `Laplace_diff` prior, which is a Laplace distribution on differences between neighbouring function values:

In [ ]:
x = Laplace_diff(location=np.zeros(n), scale=0.01)

We create a new Bayesian Problem:

In [ ]:
BP_lap = BayesianProblem(y, x)
print(BP_lap)

Set the data:

In [ ]:
BP_lap.set_data(y=y_data)

And rerun the `UQ` method:

In [ ]:
samples_lap = BP_lap.UQ(exact=probInfo.exactSolution)

Note how a different sampler was chosen due to the change of prior, and how the prior has changed the solution to be more similar to the exact solution.

We also plot a few selected samples and note the more piece-wise constant appearance:

In [ ]:
samples_lap.plot();

## Try yourself (optional):  
- Play with the parameters of the Laplace prior to see if a better solution can be obtained.
- Try the `Cauchy_diff` prior instead with scale parameter 0.01 (only for `Deconvolution1D`) for an even more edge-preserving prior, which however is more difficult to sample reliably.

# 5. ★ Gibbs sampling for hierarchical problems   <a class="anchor" id="Gibbs"></a> 

Until now we assumed that the mean, std and scale parameters of the prior and noise distributions were known. If these parameters are unknown, we include them as parameters to be estimated in a joint Bayesian inverse problem. Only very few additional lines of CUQIpy are needed to achieve this, through a hierarchical prior.

For example we can assume weakly informative Gamma distribution prior distributions on $\sigma_x$ and $\sigma_e$ such that we have the four random variables:
$$
\begin{align*}
d_x  &\sim \mathrm{Gamma}(1, 10^{-2}), \\
d_e  &\sim \mathrm{Gamma}(1, 10^{-2}), \\
\mathbf{x}&\sim\mathcal{N}(\mathbf{0},d_x^{-1}\mathbf{I}_n), \\
\mathbf{y}&\sim\mathcal{N}(\mathbf{Ax}, d_e^{-1}\mathbf{I}_n).
\end{align*}
$$

We specify the four distributions in CUQIpy. To specify the hierarchical structure we use python lambda functions (equivalent to anonymous functions in MATLAB):

In [ ]:
d_x = Gamma(1, 0.01)
d_e = Gamma(1, 0.01)
x = GaussianCov(np.zeros(n), cov=lambda d_x : 1/d_x)
y = GaussianCov(A@x, cov=lambda d_e : 1/d_e)

and proceed as before to specify the Bayesian Problem, now with the four distributions:

In [ ]:
BP = BayesianProblem(y, x, d_x, d_e)
print(BP)

Again, we provide the observed data for `y`:

In [ ]:
BP.set_data(y=y_data)

As before, we can run the UQ method, which will use a Gibbs sampler automatically selecting suitable sampling methods for each of the three parameters to be sampled `x`, `d_x` and `d_e`. We provide the true solution for `x` and the true precision for the noise for comparison in the generated plots:

In [ ]:
samplesH = BP.UQ(Ns=2000, exact={"x":probInfo.exactSolution, "d_e":d_e_true})

Earlier we saw that the Laplace_diff prior gave a more piecewise constant behavior of `x`and we can also try employing that as part of the hierarchical problem:

In [ ]:
x  = Laplace_diff(np.zeros(n), lambda d_x: 1/d_x)

We set up the Bayesian Problem and specify the observed data for `y`:

In [ ]:
BP = BayesianProblem(y, x, d_x, d_e)
BP.set_data(y=y_data)

And we sample it using the `UQ` method, this time returning the generated samples:

In [ ]:
samplesH = BP.UQ(Ns=2000, exact={"x":probInfo.exactSolution, "d_e":d_e_true})

We see the effect of the Laplace_diff prior once again on the solution, and in addition we see trace plots for the `d_x` and `d_e` parameters produced by the `UQ` method with the Gibbs sampler.

Having returned the samples we can pick out the individual parameters and do separate plots manually:

In [ ]:
samplesH["x"].plot();

In [ ]:
samplesH["d_x"].plot_trace();